In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
from numpy import loadtxt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance
import math
import random
import time
from heapq import nsmallest

In [2]:
df_csv = pd.read_csv(' ') #Read data set file

,Index,demand,prob_fail,fixed cost,lat,lon
0,0,297.60021,0.067,115800,38.566850,121.467360
1,1,179.90455,0.072,101800,42.665750,73.799017
2,2,169.86510,0.067,72600,30.305880,97.750522
3,3,129.37926,0.068,72400,30.457000,84.281399
4,4,118.81643,0.043,38400,40.276050,76.884503
5,5,114.30602,0.050,59200,39.781433,89.644654
6,6,108.47115,0.005,66000,39.988933,82.987381
7,7,92.95297,0.054,48400,42.709100,84.553996
8,8,77.30188,0.062,71300,40.223400,74.764224
9,9,66.28637,0.069,96600,35.821950,78.658753


In [3]:
def lat_lon_distmatrix(df_csv):
    dist = np.zeros((length,length+1)) #Plus emergency 
    df = pd. DataFrame(df_csv[['lat','lon']])
    records = df.to_records(index=False)
    result = list(records)
    for i in range(len(result)):
        for j in range(len(result)):
            dist[i,j] = geopy.distance.distance(result[i], result[j]).km*0.621371
    return dist
def euc_distmatrix(df_csv):
    dist = np.zeros((length,length+1)) #Plus emergency 
    df = pd. DataFrame(df_csv[['lat','lon']])
    records = df.to_records(index=False)
    result = list(records)
    for i in range(len(result)):
        for j in range(len(result)):
            dist[i,j] = math.sqrt( (result[i][0]- result[j][0])**2 + (result[i][1]- result[j][1])**2)
    return dist
    
length = df_csv.shape[0]
c_ij = lat_lon_distmatrix(df_csv)*(10**(-5))
c_ij[:,length] = c_ij[:,length]+0.1  #Emergency cost

In [9]:
V_1 = [i for i in range(0, length)] # Customer
V_2 = [i for i in range(0, length+1)] # Facility + Emergency
R = [i for i in range(0, 3)] #Backup level
T = [i for i in range(1, 5)] #Time 4 period  (1,5)
TE = [i for i in range(0, 5)] # include period 0 (0,5)


##########Initial node at period 0############ 
# 49 nodes data set
#idx_x = [0,2,4,7,21,29]#
# 150 nodes data set
#idx_x = [0,1,2,7,34,37,136]# 
# 88 nodes data set
#idx_x = [3,4,6,16,29,32,45,58,66]# 

set_init = set([0,2,4,7,21,29]) #initial node
##############################################


d_i = list(df_csv['demand']*10000)

f_j = list(df_csv['fixed cost']) + [0]

A = [(i, j, r, t) for i in V_1 for j in V_2 for r in R for t in T]


J = V_2[-1]
E = [(i, j) for i in V_1 for j in V_2 ]

A_1 = [(i, j, r, t) for i in V_1 for j in V_2[:-1] for r in R[:-1] for t in T]

o = 3000
c = 3000
jt = [(j, t) for j in V_2 for t in T]

## Multi period information #####################
q_jt = np.zeros((len(V_1)+1,4))
info = [1,2,4,8]
for i in range(0,4):
    q_jt[:,i] = list(df_csv['prob_fail'] *(info[i])) + [0] #Prob_fail + Emergency facility
    # (i+1)*0.25


d_it = np.zeros((len(d_i),4))
#################################################
rho = 0.5 #0, 0.5, 1 
#################################################
for i in range(0,4):
    d_it[:,i] = np.array(d_i)*((q_jt[:-1,i]*rho)+1) #Prob_fail + Emergency facility

    
for i in V_1:
    #if i not in idx_fix:      
    f_j[i] =  f_j[i]/10
##################################################



In [12]:
# initial location at period 0

idx_x = list(set_init)
X = np.zeros(len(V_1))
for i in idx_x:
    X[i] = 1.0
    

# Optimization model

In [ ]:
m = gp.Model("LRUFL")

#x = m.addVars(jt, vtype=GRB.BINARY, name ="x")
y = m.addVars(A,  vtype=GRB.BINARY, name ="y")
p = m.addVars(A, vtype=GRB.CONTINUOUS, name ="p")
w = m.addVars(A, vtype=GRB.CONTINUOUS, name ="w")
z_o = m.addVars(jt, vtype=GRB.BINARY, name ="z_o")
z_c = m.addVars(V_2, TE, vtype=GRB.BINARY, name ="z_c") #include 0 period bd obj function
m.update()

m.setObjective(quicksum(w[i, j, r, t]*c_ij[i, j]*d_it[i,t-1] for i,j,r,t in A) +  
               quicksum(f_j[j]* (X[j]+  quicksum(z_o[j , te] for te in T[:t]) - 
                                 quicksum(z_c[j , te] for te in TE[:t] ) ) for j in V_2[:-1] for t in T) + 
               quicksum(o*z_o[j, t] for j, t in jt) +
               quicksum(c*z_c[j, t] for j in V_2 for t in TE[:-1]), #Model can close end of period zero TE[:-1]
               GRB.MINIMIZE)



# Constraint foe z_c #Force period zero cannot be closed
#m.addConstrs( z_c[j, 0]   == 0
#             for j in V_2)

# Constraint 1b
m.addConstrs( quicksum(y[i, j, r, t] for j in V_2[:-1]) + quicksum(y[i, J, s, t] for s in range(r+1))  == 1 
             for i in V_1 for r in R for t in T)

# Constraint 1c REVISED from x[j ,t]
m.addConstrs( quicksum(y[i, j, r, t] for r in R[:-1])  <= X[j]+  quicksum(z_o[j , te] for te in T[:t]) - quicksum(z_c[j , te] for te in TE[:t] )
             for i in V_1 for j in V_2[:-1] for t in T )

# Constraint 1d
m.addConstrs( quicksum(y[i, J, r, t] for r in R)  == 1
             for i in V_1 for t in T)

# Constraint 1e 
m.addConstrs( p[i, j, r, t]  == 1-q_jt[j, t-1]
             for i in V_1 for j in V_2 for r in [0] for t in T  )

# Constraint 1f (revise)
m.addConstrs( p[i, j, r, t]  == ( 1-q_jt[j, t-1] )*quicksum(q_jt[k, t-1]*w[i, k, r-1, t]/(1-q_jt[k, t-1]) for k in V_2[:-1])
             for i in V_1 for j in V_2 for r in R[1:] for t in T )

# Constraint 2a 
m.addConstrs( w[i, j, r, t]  <= p[i, j, r, t]
             for i in V_1 for j in V_2 for r in R for t in T)
# Constraint 2b 
m.addConstrs( w[i, j, r, t]  <= y[i, j, r, t]
             for i in V_1 for j in V_2 for r in R for t in T)
# Constraint 2c
m.addConstrs( w[i, j, r, t]  >= 0
             for i in V_1 for j in V_2 for r in R for t in T)
# Constraint 2d
m.addConstrs( w[i, j, r, t]  >= p[i, j, r, t]+y[i, j, r, t]-1
             for i in V_1 for j in V_2 for r in R for t in T)

m.addConstrs((X[j]+  quicksum(z_o[j , te] for te in T[:t]) - 
                                 quicksum(z_c[j , te] for te in TE[:t] ) ) >= 0 
             for j in V_2[:-1] for t in T )
m.addConstrs((X[j]+  quicksum(z_o[j , te] for te in T[:t]) - 
                                 quicksum(z_c[j , te] for te in TE[:t] ) ) <= 1
             for j in V_2[:-1] for t in T )

# Constraint 1g x, y are Binary
#m.addConstrs( quicksum(x[j] for j in V_2)  >= 1 for j in V_2) # open more than 1

m.setParam('MIPGap', 0.1)
m.setParam('TimeLimit', 3600)
m.optimize()

Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 157780 rows, 88650 columns and 1324764 nonzeros
Model fingerprint: 0xf1992314
Variable types: 58800 continuous, 29850 integer (29850 binary)
Coefficient statistics:
  Matrix range     [4e-03, 3e+00]
  Objective range  [2e+01, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+00]
Presolve removed 122892 rows and 58662 columns
Presolve time: 1.87s
Presolved: 34888 rows, 29988 columns, 168707 nonzeros
Variable types: 10045 continuous, 19943 integer (19943 binary)
Found heuristic solution: objective 3997925.6251
Found heuristic solution: objective 1469729.2616

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 3.198958e+05, 62

H  268   273                    514220.87098 332463.389  35.3%   740  102s
H  308   328                    508613.54574 332463.389  34.6%   688  106s
H  390   410                    508462.67024 332463.389  34.6%   577  109s
H  398   410                    508367.75745 332463.389  34.6%   569  109s
   409   471 338633.473   65 4913 508367.757 332463.389  34.6%   559  110s
H  470   477                    508365.98727 332463.389  34.6%   507  117s
H  471   477                    508352.69033 332463.389  34.6%   506  117s
H  471   477                    507319.60547 332463.389  34.5%   506  117s
H  472   477                    506443.08387 332463.389  34.4%   505  117s
H  475   477                    505630.77803 332463.389  34.2%   502  117s
   493   533 338935.988   75 4496 505630.778 332463.389  34.2%   490  121s
   557   585 340440.260   83 4216 505630.778 332463.389  34.2%   487  125s
H  632   667                    505617.19602 332463.389  34.2%   471  129s
H  651   667             

  4221  2730 418918.360   45 4792 503384.006 417435.497  17.1%   376  637s
  4228  2735 419333.454   45 4016 503384.006 418565.707  16.8%   379  641s
  4235  2741 419628.359   46 4450 503384.006 418565.707  16.8%   383  645s
  4252  2755 419650.924   47 4421 503384.006 418910.394  16.8%   387  653s
  4263  2759 420210.262   48 3954 503384.006 418910.394  16.8%   390  658s
  4273  2765 428258.862   48 3986 503384.006 418910.394  16.8%   397  664s
  4284  2774 420356.705   49 3821 503384.006 418910.394  16.8%   400  668s
  4301  2780 421876.013   50 4544 503384.006 418910.394  16.8%   402  672s
  4312  2785 422938.324   50 3878 503384.006 418910.394  16.8%   404  677s
  4321  2793 422228.054   51 4758 503384.006 418910.394  16.8%   408  685s
  4332  2806 422869.745   52 4609 503384.006 418910.394  16.8%   412  690s
  4365  2826 424169.786   54 4448 503384.006 418910.394  16.8%   418  699s
  4380  2841 429656.280   54 4065 503384.006 418910.394  16.8%   422  704s
  4400  2857 424204.346  

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\ipykernel\iostream.py", line 518, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


In [191]:
for z in z_o.keys():
    if z_o[z].x > 0:
        print(f"\n Z open values {z}")
        
for z in z_c.keys():
    if z_c[z].x > 0:
        print(f"\n Z close values {z}")

        



 Z open values (6, 1)

 Z open values (48, 1)

 Z close values (7, 0)
